In [1]:
import config
from utils import get_chromedriver, get_seduta


import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from bs4 import BeautifulSoup
from soup2dict import convert
import pandas as pd
import numpy as np
import xlsxwriter

In [2]:
CHROMEDRIVER_PATH = r"C:\Users\Alessandro Bitetto\Downloads\UniPV\ICOs\WebDriver\chromedriver"

In [3]:
GRADUATION_DATE = '17/07/2023'
ESSE3_URL = "https://studentionline.unipv.it/esse3/Home.do"
ROOT_URL = "https://studentionline.unipv.it/"    # used to open link inside esse3
CHECK_YEAR = '2013/2014'    # if anno iscrizione is before CHECK_YEAR, corresponding cell is filled in yellow

# Download Lista Studenti

In [4]:
# get seduta session
driver=get_seduta(CHROMEDRIVER_PATH, GRADUATION_DATE, ESSE3_URL, ROOT_URL, config)
wait=WebDriverWait(driver, 10)

# get commissione
time.sleep(1)
commissione=wait.until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[1]")))
commissione=convert(BeautifulSoup(commissione.get_attribute('outerHTML'), 'html.parser'))
comm_list=commissione['table'][0]['tbody'][0]['tr']
df_commissione=pd.DataFrame()
for i, com in enumerate(comm_list):
    df_commissione=pd.concat([df_commissione,
                              pd.DataFrame({'Docente': com['td'][0]['#text'], 'Ruolo': com['td'][1]['#text']}, index=[i+1])])
print(f'\n- Found {len(comm_list)} in commissione')
display(df_commissione)
file_name='Seduta Laurea '+GRADUATION_DATE.replace('/', '_')+'_commissione.csv'
df_commissione.to_csv(file_name, index=False, sep=';')
print('\nFile saved in', file_name)
    
# get student list
try:
#     driver.execute_script(f"window.scrollTo(0, {600});")   # scroll down to let the table appear
    time.sleep(1)
    studenti=wait.until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]")))
    studenti=convert(BeautifulSoup(studenti.get_attribute('outerHTML'), 'html.parser'))
    stud_list=studenti['table'][0]['tbody'][0]['tr']
    print(f'\n\n\n- Found {len(stud_list)} students')
except:
    print('\n\n\n## Error for lista studenti')
    raise


- Found 6 in commissione


,Docente,Ruolo
1,RIZZO MARIO VALENTINO,Membro Effettivo
2,DENICOLAI STEFANO,Membro Effettivo
3,TARANTOLA CLAUDIA,Membro Effettivo
4,ZUCCHELLA ANTONELLA,Membro Effettivo
5,BITETTO ALESSANDRO,Segretario
6,TIRELLI PATRIZIO,Presidente



File saved in Seduta Laurea 17_07_2023_commissione.csv



- Found 9 students


In [5]:
df_studenti=pd.DataFrame()
driver.switch_to.window(driver.window_handles[0])
for i, st in enumerate(stud_list):
    
    print(f"Reading student {i+1}/{len(stud_list)}", end = "\r")
    
    link=st['td'][10]['a'][0]['@href']
    driver.execute_script(f"window.open('{link}')")
    new_tab= driver.window_handles[i+1]
    driver.switch_to.window(new_tab)

    # get anno iscrizione
    iscrizione=wait.until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[1]")))
    iscrizione=convert(BeautifulSoup(iscrizione.get_attribute('outerHTML'), 'html.parser'))
    anno_iscrizione=[]
    for el in iscrizione['table'][0]['tbody'][0]['tr']:
        anno_iscrizione.append(el['#text'])

    # get punti tesi
    media=wait.until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/div[3]")))
    media=convert(BeautifulSoup(media.get_attribute('outerHTML'), 'html.parser'))
    crediti=media['div'][0]['dl'][0]['dd'][3]['#text'].replace('\xa0', '').replace('\u200b', '')
    crediti_tesi=media['div'][0]['dl'][0]['dd'][4]['#text'].replace('\xa0', '').replace('\u200b', '')
    media_pesata=media['div'][0]['dl'][0]['dd'][5]['#text'].replace('\xa0', '').replace('\u200b', '')

    # get voto proposto
    voto=wait.until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/div[4]")))
    voto=convert(BeautifulSoup(voto.get_attribute('outerHTML'), 'html.parser'))
    df_voto=pd.DataFrame(index=[i])
    for ind, el in enumerate(voto['div'][0]['dl'][0]['dt']):
        el_name=el['#text']
        el_val=voto['div'][0]['dl'][0]['dd'][ind]['navigablestring']
        el_val='\n'.join(el_val).replace('\xa0', '').replace('\u200b', '')
        if el_name == 'Voto proposto':
            el_val=int(el_val) 
        df_voto[el_name]=[el_val]
        
    add_row=pd.concat([pd.DataFrame({'Nome': st['td'][0]['#text'],
                                     'Matricola': st['td'][1]['#text'],
                                     'Nascita': st['td'][2]['#text'],
                                     'Corso': st['td'][3]['#text'],
                                     'Titolo tesi': st['td'][4]['#text'],
                                     'Relatore': st['td'][5]['#text'],
                                     'Anno iscrizione': '\n'.join(anno_iscrizione),
                                     'Crediti': crediti,
                                     'Crediti per tesi': crediti_tesi,
                                     'Media pesata': media_pesata.replace(' - ', '\n'),
                                     'link': link}, index=[i]),
                       df_voto], axis=1)

    df_studenti=pd.concat([df_studenti, add_row])
    df_studenti['Punti tesi']=''
    df_studenti['Lode']=''
    df_studenti['Encomio']=''
    df_studenti['Voto Finale']=0
    move_col = df_studenti.pop('link')
    df_studenti.insert(df_studenti.shape[1], 'link', move_col)
df_studenti

,Nome,Matricola,Nascita,Corso,Titolo tesi,Relatore,Anno iscrizione,Crediti,Crediti per tesi,Media pesata,...,Punteggio aggiuntivo,Motivazione punteggio aggiuntivo,Punteggio proposto,Motivazione punteggio proposto,Richiesta di eccellenza,Punti tesi,Lode,Encomio,Voto Finale,link
0,CAPASSO CARLA,493624,01/10/1998,"ECONOMICS, FINANCE AND INTERNATIONAL INTEGRATI...",The performance of Mutual Funds: a view on the...,TIRELLI PATRIZIO,1 2020/2021 In corso\n2 2021/2022 In corso\n2 ...,96,24,95.688/110 \n26.097/30,...,0.0,Ottenuti 0.00 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
1,IETTI CLAUDIA,490105,16/03/1997,"ECONOMICS, FINANCE AND INTERNATIONAL INTEGRATI...",ANALISI DEL MERCATO IMMOBILIARE NELLA CITTA ME...,TIRELLI PATRIZIO,1 2020/2021 In corso\n2 2021/2022 In corso\n2 ...,96,24,101.72/110 \n27.742/30,...,0.6,Ottenuti 0.60 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
2,SADASIVAM SHRUTHE,493357,25/09/1996,INTERNATIONAL BUSINESS AND ENTREPRENEURSHIP - ...,Factors Influencing Consumer Purchasing Choice...,ZUCCHELLA ANTONELLA,1 2020/2021 In corso\n2 2021/2022 In corso\n2 ...,96,24,87.542/110 \n23.875/30,...,0.0,Ottenuti 0.00 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
3,HRICOVA KATARINA,509264,19/03/1999,"ECONOMICS, FINANCE AND INTERNATIONAL INTEGRATI...",COMPARATIVE ANALYSIS OF FUNDING FROM EUROPEAN ...,TARANTOLA CLAUDIA,1 2021/2022 In corso\n2 2022/2023 In corso,96,24,93.323/110 \n25.452/30,...,0.3,Ottenuti 0.30 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
4,KOPTÁKOVÁ MARTINA,506413,11/12/1996,"ECONOMICS, FINANCE AND INTERNATIONAL INTEGRATI...",GREEN TOTAL FACTOR PRODUCTIVITY AND \nECONOMIC...,TARANTOLA CLAUDIA,1 2021/2022 In corso\n2 2022/2023 In corso,96,24,93.559/110 \n25.516/30,...,0.0,Ottenuti 0.00 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
5,LÍŠKOVÁ MIRIAM,506415,14/06/1999,"ECONOMICS, FINANCE AND INTERNATIONAL INTEGRATI...",FINANCIAL ANALYSIS OF A SELECTED COMPANY,TARANTOLA CLAUDIA,1 2021/2022 In corso\n2 2022/2023 In corso,96,24,96.161/110 \n26.226/30,...,0.3,Ottenuti 0.30 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
6,CORDINI ENRICO,482587,01/09/1997,ECONOMIA E GESTIONE DELLE IMPRESE,"L'Eni fra geopolitica, energia ed ecologia: da...",RIZZO MARIO VALENTINO,1 2019/2020 In corso\n2 2020/2021 In corso\n2 ...,96,24,88/110 \n24/30,...,0.0,Ottenuti 0.00 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
7,STUNDOVA ALEXANDRA,506471,18/04/1999,"ECONOMICS, FINANCE AND INTERNATIONAL INTEGRATI...",STRENGTHS AND WEAKNESSES OF THE COMPETITIVENES...,TARANTOLA CLAUDIA,1 2021/2022 In corso\n2 2022/2023 In corso,96,24,90.129/110 \n24.581/30,...,0.0,Ottenuti 0.00 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
8,SIVAKUMAR MANOJ KUMAR,493404,23/12/1996,INTERNATIONAL BUSINESS AND ENTREPRENEURSHIP - ...,Gestione strategica del marchio per la ricerca...,DENICOLAI STEFANO,1 2020/2021 In corso\n2 2021/2022 In corso\n2 ...,96,24,82.156/110 \n22.406/30,...,0.0,Ottenuti 0.00 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...


In [6]:
file_name='Seduta Laurea '+GRADUATION_DATE.replace('/', '_')+'.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df_studenti.to_excel(writer, sheet_name='Laurea', index=False)

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Laurea']

wrap_format = workbook.add_format({'text_wrap' : True, 'align': 'top'})
top_format=workbook.add_format({'align': 'top'})
yellow_format=workbook.add_format({'text_wrap' : True, 'align': 'top', 'bg_color': 'yellow'})
worksheet.set_column(0, 5, 15, top_format)
worksheet.set_column(7, 9, 15, top_format)
worksheet.set_column(11, 11, 20, top_format)
worksheet.set_column(6, 6, 20, wrap_format)
worksheet.set_column(10, 10, 20, wrap_format)
worksheet.set_column(12, 12, 30, wrap_format)

# add formula for voto finale
col_num=df_studenti.columns.get_loc("Voto Finale")+1
col_letter_finale=chr(ord('@')+col_num)
col_num=df_studenti.columns.get_loc("Voto proposto")+1
col_letter_proposto=chr(ord('@')+col_num)
col_num=df_studenti.columns.get_loc("Punti tesi")+1
col_letter_punti=chr(ord('@')+col_num)
for ind, row in df_studenti.iterrows():
    worksheet.write_formula(col_letter_finale+str(ind+2), '='+ col_letter_proposto+str(ind+2) +' + '+ col_letter_punti+str(ind+2))
    # https://xlsxwriter.readthedocs.io/working_with_formulas.html
    
# fill cell with yellow if "Anno iscrizione" is before CHECK_YEAR
col_num=df_studenti.columns.get_loc("Anno iscrizione")+1
col_letter=chr(ord('@')+col_num)
for ind, row in df_studenti.iterrows():
    anno=[]
    for an in row['Anno iscrizione'].split('\n'):
        anno.append([int(x.split('/')[0]) for i, x in enumerate(an.split(' ')) if '/' in x][0])
    if min(anno) <= int(CHECK_YEAR.split('/')[0]):
        worksheet.write(col_letter+str(ind+2), row['Anno iscrizione'], yellow_format)

writer.save()

print('\nFile saved in', file_name)


File saved in Seduta Laurea 17_07_2023.xlsx


C:\Users\Alessandro Bitetto\AppData\Local\Temp\ipykernel_13956\1437380815.py:42: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


# Upload Voti

In [4]:
df_commissione=pd.read_csv('Seduta Laurea '+GRADUATION_DATE.replace('/', '_')+'_commissione.csv', sep=';')
df_studenti=pd.read_excel('Seduta Laurea '+GRADUATION_DATE.replace('/', '_')+'.xlsx')
df_studenti['Lode']=df_studenti['Lode'].fillna('')
df_studenti['Encomio']=df_studenti['Encomio'].fillna('')

In [5]:
# get seduta session
driver=get_seduta(CHROMEDRIVER_PATH, GRADUATION_DATE, ESSE3_URL, ROOT_URL, config)
wait=WebDriverWait(driver, 10)

In [6]:
driver.switch_to.window(driver.window_handles[0])

for i, row in df_studenti.iterrows():
    
    print(f"Uploading student {i+1}/{len(df_studenti)}", end = "\r")
    
    link=ROOT_URL+row['link']
    driver.execute_script(f"window.open('{link}')")
    new_tab= driver.window_handles[i+1]
    driver.switch_to.window(new_tab)
    
    # input punti tesi
    try:
        punti_tesi=wait.until(
                        EC.presence_of_element_located((By.ID, 'grad-dettLau-puntiTesi')))
        punti_tesi.click()
        punti_tesi.clear()
        punti_tesi.send_keys(row['Punti tesi'])
        wait.until(EC.presence_of_element_located((By.ID, 'grad-dettLau-annotazioni'))).click()
    except:
        print(f'\n## Error for punti tesi {row["Nome"]} ({i+1})')

    # tick Lode and Encomio
    if row['Lode'] != '':
        try:
            lode=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-lode1')))
            lode.click()
        except:
            print(f'\n## Error for Lode {row["Nome"]} ({i+1})')
    if row['Encomio'] != '':
        try:
            encomio=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-encomio1')))
            encomio.click()
        except:
            print(f'\n## Error for Encomio {row["Nome"]} ({i+1})')

    # tick Commissione
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        commissione=wait.until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/form/div[3]")))
        commissione=convert(BeautifulSoup(commissione.get_attribute('outerHTML'), 'html.parser'))
        get_checkbox_id=commissione['div'][0]['div'][0]['@id'].replace('gbox', 'cb')
        all_commissione=wait.until(
                        EC.presence_of_element_located((By.ID, get_checkbox_id)))
        all_commissione.click()
    except:
        print(f'\n## Error for Commissione {row["Nome"]} ({i+1})')

    # save
    try:
        save=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-btnSubmit')))
        save.click()
    except:
        print(f'\n## Error for Save {row["Nome"]} ({i+1})')

    save and exit
    try:
        save_exit=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-btnSubmitExit')))
        save_exit.click()
    except:
        print(f'\n## Error for Save and Exit {row["Nome"]} ({i+1})')


# check final grade mismatch
driver.switch_to.window(driver.window_handles[0])
final_grade=wait.until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]")))
final_grade=convert(BeautifulSoup(final_grade.get_attribute('outerHTML'), 'html.parser'))

for i, el in enumerate(final_grade['table'][0]['tbody'][0]['tr']):
    grade_evaluated=int(el['td'][8]['#text'])
    matricola=int(el['td'][1]['#text'])
    name=el['td'][0]['#text']
    grade_expected=df_studenti[df_studenti['Matricola'] == matricola]['Voto Finale'].values[0]
    if grade_evaluated != grade_expected:
        print(f'\n## Expected Votazione Finale mismatch: {name} ({i+1})')
        
print('\nVotazione Finale: Check Done')

Uploading student 9/9
Votazione Finale: Check Done
